In [3]:
%pip install requests
%pip install beautifulsoup4
%pip install pandas


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: /usr/local/Cellar/jupyterlab/4.0.5/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: /usr/local/Cellar/jupyterlab/4.0.5/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 26.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 33.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 7.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 27.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: /usr/local/Cellar/jupyterlab/4.0.5/libexec/bin/python -m pip install --upgrade pip
Note: you may n

In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Define the URL
url = 'https://www.scdf.gov.sg/firecode/table-of-content'

# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content of the page
soup = BeautifulSoup(response.content, 'html.parser')

# Find all accordion elements
accordions = soup.find_all('div', class_='firecode-accordion')

# Initialize lists to store data
chapters = []
clauses = []
contents = []

# Iterate through each accordion
for accordion in accordions:
    # Extract chapter title
    chapter_title = accordion.find('h5').text.strip()

    # Extract clauses within the accordion
    clause_elements = accordion.find_all('div', class_='clause-content-block')

    # Iterate through each clause
    for clause in clause_elements:
        # Extract individual clauses and their content
        for link in clause.find_all('a'):
            clause_title = link.text.strip()
            clause_link = link['href']

            # Append data to lists
            chapters.append(chapter_title)
            clauses.append(clause_title)
            contents.append(clause_link)

# Create a DataFrame from the lists
firecode_df = pd.DataFrame({'Chapter': chapters, 'Clause': clauses, 'Content': contents})

# Display the DataFrame
print(firecode_df.head())


                       Chapter  \
0          Chapter 1 - General   
1          Chapter 1 - General   
2          Chapter 1 - General   
3  Chapter 2 - Means of Escape   
4  Chapter 2 - Means of Escape   

                                           Clause  \
0                               Clause 1.1  Scope   
1                 Clause 1.2  Codes and Standards   
2                        Clause 1.3 Abbreviations   
3                              Clause 2.1 General   
4  Clause 2.2 Determinations of Exit Requirements   

                                             Content  
0  /firecode/table-of-content/chapter-1-general/c...  
1  /firecode/table-of-content/chapter-1-general/c...  
2  /firecode/table-of-content/chapter-1-general/c...  
3  /firecode/table-of-content/chapter-2-means-of-...  
4  /firecode/table-of-content/chapter-2-means-of-...  


In [17]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By

# Install Selenium
# Uncomment the line below if you haven't installed Selenium yet
# !pip install selenium

# Download ChromeDriver from https://sites.google.com/chromium.org/driver/
# Specify the path to the ChromeDriver executable
# chrome_driver_path = '/path/to/chromedriver'
chrome_driver_path = '/Users/lmchak/llm/chromedriver-mac-x64/chromedriver'


# Set up the Chrome WebDriver
driver = webdriver.Chrome(executable_path=chrome_driver_path)

# URL of the main page
url = "https://www.scdf.gov.sg/firecode/table-of-content"


# Open the main page
driver.get(url)

# Find all the chapter links
chapter_links = driver.find_elements(By.CSS_SELECTOR, ".firecode-accordion__controller--wrapper a")

# Create an empty DataFrame
firecode_df = pd.DataFrame(columns=['Chapter', 'Clause', 'Content'])

# Loop through each chapter link
for chapter_link in chapter_links:
    chapter_title = chapter_link.text.strip()

    # Click on the chapter link to reveal the clauses
    chapter_link.click()

    # Find all the clause links under the chapter
    clause_links = driver.find_elements(By.CSS_SELECTOR, ".firecode-accordion__content a")

    # Loop through each clause link
    for link in clause_links:
        clause_title = link.text.strip()
        clause_link = link.get_attribute('href')

        # Append data to DataFrame
        firecode_df = firecode_df.append({
            'Chapter': chapter_title,
            'Clause': clause_title,
            'Content': clause_link
        }, ignore_index=True)

# Close the WebDriver
driver.quit()

# Display the DataFrame
print("Unique Chapters:", firecode_df['Chapter'].unique())
print("\nDataFrame:")
print(firecode_df)


TypeError: WebDriver.__init__() got an unexpected keyword argument 'executable_path'

In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import pandas as pd

# Set up the Chrome WebDriver
chrome_driver_path = '/Users/lmchak/llm/chromedriver-mac-x64/chromedriver'
chrome_service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=chrome_service)

# URL of the main page
url = "https://www.scdf.gov.sg/firecode/table-of-content"

# Navigate to the main page
driver.get(url)

# Extracting data
firecode_data = []

# Find all chapter links
chapter_links = driver.find_elements(By.XPATH, "//div[@class='firecode-accordion']/div[@class='firecode-accordion__controller']/div[@class='firecode-accordion__controller--wrapper']/h5/a")

for chapter_link in chapter_links:
    chapter_title = chapter_link.text.strip()
    chapter_content_link = chapter_link.get_attribute("href")

    # Append chapter data
    firecode_data.append({
        'Chapter': chapter_title,
        'Clause': None,
        'Content': chapter_content_link
    })

# Create a DataFrame
firecode_df = pd.DataFrame(firecode_data)

# Display DataFrame
print("DataFrame:")
print(firecode_df)

# Close the WebDriver
driver.quit()


DataFrame:
                                              Chapter Clause  \
0                                 Chapter 1 - General   None   
1                         Chapter 2 - Means of Escape   None   
2             Chapter 3 - Structural Fire Precautions   None   
3   Chapter 4 - Site Planning & External Firefight...   None   
4               Chapter 5 - Electrical Power Supplies   None   
5                    Chapter 6 - Firefighting Systems   None   
6   Chapter 7 - Mechanical Ventilation & Smoke Con...   None   
7   Chapter 8 - Emergency Lighting & Voice Communi...   None   
8   Chapter 9 - Additional Requirements for Each P...   None   
9   Chapter 10 - Requirements for Special Installa...   None   
10  Chapter 11 - Regulated Fire Safety Products an...   None   
11                                        Appendix 01   None   
12       Appendix 02 - Fire Safety Instruction Manual   None   

                                              Content  
0   https://www.scdf.gov.sg/firecode

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time

def get_links_recursive(driver, url, links=[]):
    # Navigate to the URL
    driver.get(url)
    time.sleep(2)  # Add a delay to ensure the page is loaded
    
    # Find all links on the current page
    page_links = driver.find_elements(By.XPATH, "//a[@href]")
    
    # Append the links to the list
    for page_link in page_links:
        link = page_link.get_attribute("href")
        links.append(link)
    
    # Find and follow links to sub-pages
    sub_pages = driver.find_elements(By.XPATH, "//div[@class='firecode-accordion__content']//a[@href]")
    for sub_page in sub_pages:
        sub_page_link = sub_page.get_attribute("href")
        if sub_page_link not in links:
            links = get_links_recursive(driver, sub_page_link, links)
    
    return links

# Set up the Chrome WebDriver
chrome_driver_path = '/Users/lmchak/llm/chromedriver-mac-x64/chromedriver'  # Replace with the actual path to your chromedriver
chrome_service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=chrome_service)

# Start the recursive link retrieval
main_url = "https://www.ura.gov.sg/Corporate/Guidelines/Development-Control/Non-Residential/C-CI/Introduction"
all_links = get_links_recursive(driver, main_url)

# Display the retrieved links
for i, link in enumerate(all_links, start=1):
    print(f"{i}. {link}")

# Close the WebDriver
driver.quit()


1. https://www.ura.gov.sg/Corporate
2. javascript:__doPostBack('header_0$LinkButton1','')
3. https://www.ura.gov.sg/Corporate/About-Us
4. https://www.ura.gov.sg/Corporate/About-Us/Who-We-Are
5. https://www.ura.gov.sg/Corporate/About-Us/What-We-Do
6. https://www.ura.gov.sg/Corporate/About-Us/Annual-Reports
7. https://www.ura.gov.sg/Corporate/About-Us/Financial-Statements
8. 
9. https://www.ura.gov.sg/Corporate/Subscription
10. https://go.gov.sg/uravp
11. https://www.ura.gov.sg/Corporate/Planning
12. https://www.ura.gov.sg/Corporate/Planning/Our-Planning-Process
13. https://www.ura.gov.sg/Corporate/Planning/Our-Planning-Process/Digitalisation
14. https://www.ura.gov.sg/Corporate/Planning/Our-Planning-Process/Long-Term-Planning
15. https://www.ura.gov.sg/Corporate/Planning/Our-Planning-Process/Bringing-plans-to-Reality
16. https://www.ura.gov.sg/Corporate/Planning/Long-Term-Plan-Review
17. https://www.ura.gov.sg/Corporate/Planning/Long-Term-Plan-Review/Public-Engagement-Journey
18. https:

In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
from urllib.parse import urljoin, urlparse

def get_relative_links(driver, url, excluded_keywords=[]):
    # Navigate to the URL
    driver.get(url)
    time.sleep(2)  # Add a delay to ensure the page is loaded
    
    # Find the main content div
    main_content_div = driver.find_element(By.XPATH, "//div[@id='pnlMain' and @class='mainWrap']")

    # Find all relative links within the main content
    relative_links = main_content_div.find_elements(By.XPATH, ".//a[@href]")

    # Filter out specific links
    filtered_links = []
    for rel_link in relative_links:
        href = rel_link.get_attribute("href")
        if not any(keyword in href for keyword in excluded_keywords):
            filtered_links.append(href)

    return filtered_links

# Set up the Chrome WebDriver
chrome_driver_path = '/Users/lmchak/llm/chromedriver-mac-x64/chromedriver'  # Replace with the actual path to your chromedriver
chrome_service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=chrome_service)

# Start link retrieval for the specified URL
ura_url = "https://www.ura.gov.sg/Corporate/Guidelines"
excluded_keywords_ura = []  # Add any specific keywords to exclude if needed
ura_links = get_relative_links(driver, ura_url, excluded_keywords_ura)

# Display the retrieved links
for i, link in enumerate(ura_links, start=1):
    print(f"{i}. {link}")

# Close the WebDriver
driver.quit()


1. https://www.ura.gov.sg/Corporate
2. https://www.ura.gov.sg/Corporate/Guidelines
3. https://www.ura.gov.sg/Corporate/Guidelines/Circulars
4. https://www.ura.gov.sg/Corporate/Guidelines/Development-Control
5. https://www.ura.gov.sg/Corporate/Guidelines/Urban-Design
6. https://www.ura.gov.sg/Corporate/Guidelines/Street-and-Building-Names
7. https://www.ura.gov.sg/Corporate/Guidelines/Home-Business
8. https://www.ura.gov.sg/Corporate/Guidelines/Developers
9. https://www.ura.gov.sg/Corporate/Guidelines/Conservation
10. https://www.ura.gov.sg/Corporate/Guidelines/Active-Mobility


In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
from urllib.parse import urljoin, urlparse

def get_related_links(driver, base_url, url, excluded_keywords=[]):
    # Navigate to the URL
    driver.get(url)
    time.sleep(2)  # Add a delay to ensure the page is loaded

    # Find the main content div
    main_content_div = driver.find_element(By.XPATH, "//div[@id='pnlMain' and @class='mainWrap']")

    # Find all relative links within the main content
    relative_links = main_content_div.find_elements(By.XPATH, ".//a[@href]")

    # Filter out specific links and make them absolute
    filtered_links = []
    for rel_link in relative_links:
        href = rel_link.get_attribute("href")
        absolute_link = urljoin(base_url, href)
        if absolute_link != "https://www.ura.gov.sg/Corporate" and not any(keyword in href for keyword in excluded_keywords):
            filtered_links.append(absolute_link)

    return filtered_links

# Set up the Chrome WebDriver
chrome_driver_path = '/Users/lmchak/llm/chromedriver-mac-x64/chromedriver'  # Replace with the actual path to your chromedriver
chrome_service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=chrome_service)

# Base URL
base_url = "https://www.ura.gov.sg/Corporate"

# Start link retrieval for the specified URL
ura_url = "https://www.ura.gov.sg/Corporate/Guidelines"
excluded_keywords_ura = []  # Add any specific keywords to exclude if needed
ura_links = get_related_links(driver, base_url, ura_url, excluded_keywords_ura)

# Display the retrieved links
for i, link in enumerate(ura_links, start=1):
    print(f"{i}. {link}")

# Close the WebDriver
driver.quit()


1. https://www.ura.gov.sg/Corporate/Guidelines
2. https://www.ura.gov.sg/Corporate/Guidelines/Circulars
3. https://www.ura.gov.sg/Corporate/Guidelines/Development-Control
4. https://www.ura.gov.sg/Corporate/Guidelines/Urban-Design
5. https://www.ura.gov.sg/Corporate/Guidelines/Street-and-Building-Names
6. https://www.ura.gov.sg/Corporate/Guidelines/Home-Business
7. https://www.ura.gov.sg/Corporate/Guidelines/Developers
8. https://www.ura.gov.sg/Corporate/Guidelines/Conservation
9. https://www.ura.gov.sg/Corporate/Guidelines/Active-Mobility


In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time

def get_related_links_recursive(driver, base_url, url, excluded_keywords=[], visited_urls=set()):
    # Navigate to the URL
    driver.get(url)
    time.sleep(2)  # Add a delay to ensure the page is loaded

    # Find the main content div
    main_content_div = driver.find_element(By.XPATH, "//div[@id='pnlMain' and @class='mainWrap']")

    # Find all relative links within the main content
    relative_links = main_content_div.find_elements(By.XPATH, ".//a[@href]")

    # Filter out specific links and make them absolute
    filtered_links = []
    for rel_link in relative_links:
        href = rel_link.get_attribute("href")
        absolute_link = urljoin(base_url, href)
        if absolute_link != base_url and not any(keyword in href for keyword in excluded_keywords):
            filtered_links.append(absolute_link)

    # Add the current URL to the set of visited URLs
    visited_urls.add(url)

    # Find and follow links to sub-pages
    sub_pages = [sub_page.get_attribute("href") for sub_page in relative_links if sub_page.get_attribute("href") not in visited_urls]
    for sub_page_url in sub_pages:
        if sub_page_url.startswith(base_url):
            filtered_links += get_related_links_recursive(driver, base_url, sub_page_url, excluded_keywords, visited_urls)

    return filtered_links

# Set up the Chrome WebDriver
chrome_driver_path = '/Users/lmchak/llm/chromedriver-mac-x64/chromedriver'  # Replace with the actual path to your chromedriver
chrome_service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=chrome_service)

# Base URL
base_url = "https://www.ura.gov.sg/Corporate"

# Start link retrieval for the specified URL
ura_url = "https://www.ura.gov.sg/Corporate/Guidelines"
excluded_keywords_ura = []  # Add any specific keywords to exclude if needed
ura_links = get_related_links_recursive(driver, base_url, ura_url, excluded_keywords_ura)

# Display the retrieved links
for i, link in enumerate(ura_links, start=1):
    print(f"{i}. {link}")

# Close the WebDriver
driver.quit()


KeyboardInterrupt: 

In [20]:
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
from urllib.parse import urljoin

def is_pdf(url):
    # Send a HEAD request to get only the headers
    response = requests.head(url)
    
    # Check if the Content-Type header indicates a PDF file
    return 'application/pdf' in response.headers.get('Content-Type', '')

def get_related_links_recursive(driver, base_url, url, excluded_keywords=[], visited_urls=set()):
    # Navigate to the URL
    driver.get(url)
    time.sleep(2)  # Add a delay to ensure the page is loaded

    # Find the main content div
    main_content_div = driver.find_element(By.XPATH, "//div[@id='pnlMain' and @class='mainWrap']")

    # Find all relative links within the main content
    relative_links = main_content_div.find_elements(By.XPATH, ".//a[@href]")

    # Filter out specific links and make them absolute
    filtered_links = []
    for rel_link in relative_links:
        href = rel_link.get_attribute("href")
        absolute_link = urljoin(base_url, href)
        if absolute_link != base_url and not any(keyword in href for keyword in excluded_keywords) and not is_pdf(absolute_link):
            filtered_links.append(absolute_link)

    # Add the current URL to the set of visited URLs
    visited_urls.add(url)

    # Find and follow links to sub-pages
    sub_pages = [sub_page.get_attribute("href") for sub_page in relative_links if sub_page.get_attribute("href") not in visited_urls]
    for sub_page_url in sub_pages:
        if sub_page_url.startswith(base_url):
            filtered_links += get_related_links_recursive(driver, base_url, sub_page_url, excluded_keywords, visited_urls)

    return filtered_links

# Set up the Chrome WebDriver
chrome_driver_path = '/Users/lmchak/llm/chromedriver-mac-x64/chromedriver'  # Replace with the actual path to your chromedriver
chrome_service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=chrome_service)

# Base URL
base_url = "https://www.ura.gov.sg/Corporate/Guidelines"

# Start link retrieval for the specified URL
ura_links = get_related_links_recursive(driver, base_url, base_url)

# Display the retrieved links
for i, link in enumerate(ura_links, start=1):
    print(f"{i}. {link}")

# Close the WebDriver
driver.quit()


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@id='pnlMain' and @class='mainWrap']"}
  (Session info: chrome=122.0.6261.94); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000108fede18 chromedriver + 4627992
1   chromedriver                        0x0000000108fe5b43 chromedriver + 4594499
2   chromedriver                        0x0000000108be3e4a chromedriver + 392778
3   chromedriver                        0x0000000108c2f41d chromedriver + 701469
4   chromedriver                        0x0000000108c2f5b1 chromedriver + 701873
5   chromedriver                        0x0000000108c73214 chromedriver + 979476
6   chromedriver                        0x0000000108c5189d chromedriver + 841885
7   chromedriver                        0x0000000108c706d9 chromedriver + 968409
8 

In [21]:
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
from urllib.parse import urljoin

def is_pdf(url):
    # Send a HEAD request to get only the headers
    response = requests.head(url)
    
    # Check if the Content-Type header indicates a PDF file
    return 'application/pdf' in response.headers.get('Content-Type', '')

def get_related_links_recursive(driver, base_url, url, excluded_keywords=[], visited_urls=set()):
    # Navigate to the URL
    driver.get(url)
    time.sleep(2)  # Add a delay to ensure the page is loaded

    # Check if the page contains a PDF file
    if is_pdf(url):
        return []

    try:
        # Find the main content div
        main_content_div = driver.find_element(By.XPATH, "//div[@id='pnlMain' and @class='mainWrap']")

        # Find all relative links within the main content
        relative_links = main_content_div.find_elements(By.XPATH, ".//a[@href]")

        # Filter out specific links and make them absolute
        filtered_links = []
        for rel_link in relative_links:
            href = rel_link.get_attribute("href")
            absolute_link = urljoin(base_url, href)
            if absolute_link != base_url and not any(keyword in href for keyword in excluded_keywords) and not is_pdf(absolute_link):
                filtered_links.append(absolute_link)

        # Add the current URL to the set of visited URLs
        visited_urls.add(url)

        # Find and follow links to sub-pages
        sub_pages = [sub_page.get_attribute("href") for sub_page in relative_links if sub_page.get_attribute("href") not in visited_urls]
        for sub_page_url in sub_pages:
            if sub_page_url.startswith(base_url):
                filtered_links += get_related_links_recursive(driver, base_url, sub_page_url, excluded_keywords, visited_urls)

        return filtered_links
    except NoSuchElementException:
        return []

# Set up the Chrome WebDriver
chrome_driver_path = '/Users/lmchak/llm/chromedriver-mac-x64/chromedriver'  # Replace with the actual path to your chromedriver
chrome_service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=chrome_service)

# Base URL
base_url = "https://www.ura.gov.sg/Corporate/Guidelines"

# Start link retrieval for the specified URL
ura_links = get_related_links_recursive(driver, base_url, base_url)

# Display the retrieved links
for i, link in enumerate(ura_links, start=1):
    print(f"{i}. {link}")

# Close the WebDriver
driver.quit()



NameError: name 'NoSuchElementException' is not defined

In [22]:
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
from urllib.parse import urljoin

def is_pdf(url):
    # Send a HEAD request to get only the headers
    response = requests.head(url)
    
    # Check if the Content-Type header indicates a PDF file
    return 'application/pdf' in response.headers.get('Content-Type', '')

def get_related_links_recursive(driver, base_url, url, excluded_keywords=[], visited_urls=set()):
    # Check if the page contains a PDF file
    if is_pdf(url):
        return []

    # Navigate to the URL
    driver.get(url)
    time.sleep(2)  # Add a delay to ensure the page is loaded

    try:
        # Find the main content div
        main_content_div = driver.find_element(By.XPATH, "//div[@id='pnlMain' and @class='mainWrap']")

        # Find all relative links within the main content
        relative_links = main_content_div.find_elements(By.XPATH, ".//a[@href]")

        # Filter out specific links and make them absolute
        filtered_links = []
        for rel_link in relative_links:
            href = rel_link.get_attribute("href")
            absolute_link = urljoin(base_url, href)
            if absolute_link != base_url and not any(keyword in href for keyword in excluded_keywords):
                filtered_links.append(absolute_link)

        # Add the current URL to the set of visited URLs
        visited_urls.add(url)

        # Find and follow links to sub-pages
        sub_pages = [sub_page.get_attribute("href") for sub_page in relative_links if sub_page.get_attribute("href") not in visited_urls]
        for sub_page_url in sub_pages:
            if sub_page_url.startswith(base_url):
                filtered_links += get_related_links_recursive(driver, base_url, sub_page_url, excluded_keywords, visited_urls)

        return filtered_links
    except NoSuchElementException:
        return []

# Set up the Chrome WebDriver
chrome_driver_path = '/Users/lmchak/llm/chromedriver-mac-x64/chromedriver'  # Replace with the actual path to your chromedriver
chrome_service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=chrome_service)

# Base URL
base_url = "https://www.ura.gov.sg/Corporate/Guidelines"

# Start link retrieval for the specified URL
ura_links = get_related_links_recursive(driver, base_url, base_url)

# Display the retrieved links
for i, link in enumerate(ura_links, start=1):
    print(f"{i}. {link}")

# Close the WebDriver
driver.quit()


NameError: name 'NoSuchElementException' is not defined

In [23]:
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException  # Import the exception
import time
from urllib.parse import urljoin

def is_pdf(url):
    # Send a HEAD request to get only the headers
    response = requests.head(url)
    
    # Check if the Content-Type header indicates a PDF file
    return 'application/pdf' in response.headers.get('Content-Type', '')

def get_related_links_recursive(driver, base_url, url, excluded_keywords=[], visited_urls=set()):
    # Check if the page contains a PDF file
    if is_pdf(url):
        return []

    # Navigate to the URL
    driver.get(url)
    time.sleep(2)  # Add a delay to ensure the page is loaded

    try:
        # Find the main content div
        main_content_div = driver.find_element(By.XPATH, "//div[@id='pnlMain' and @class='mainWrap']")

        # Find all relative links within the main content
        relative_links = main_content_div.find_elements(By.XPATH, ".//a[@href]")

        # Filter out specific links and make them absolute
        filtered_links = []
        for rel_link in relative_links:
            href = rel_link.get_attribute("href")
            absolute_link = urljoin(base_url, href)
            if absolute_link != base_url and not any(keyword in href for keyword in excluded_keywords):
                filtered_links.append(absolute_link)

        # Add the current URL to the set of visited URLs
        visited_urls.add(url)

        # Find and follow links to sub-pages
        sub_pages = [sub_page.get_attribute("href") for sub_page in relative_links if sub_page.get_attribute("href") not in visited_urls]
        for sub_page_url in sub_pages:
            if sub_page_url.startswith(base_url):
                filtered_links += get_related_links_recursive(driver, base_url, sub_page_url, excluded_keywords, visited_urls)

        return filtered_links
    except NoSuchElementException:
        return []

# Set up the Chrome WebDriver
chrome_driver_path = '/Users/lmchak/llm/chromedriver-mac-x64/chromedriver'  # Replace with the actual path to your chromedriver
chrome_service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=chrome_service)

# Base URL
base_url = "https://www.ura.gov.sg/Corporate/Guidelines"

# Start link retrieval for the specified URL
ura_links = get_related_links_recursive(driver, base_url, base_url)

# Display the retrieved links
for i, link in enumerate(ura_links, start=1):
    print(f"{i}. {link}")

# Close the WebDriver
driver.quit()


KeyboardInterrupt: 

In [24]:
import os
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
import time
from urllib.parse import urljoin, urlparse

def is_pdf(url):
    # Send a HEAD request to get only the headers
    response = requests.head(url)

    # Check if the Content-Type header indicates a PDF file
    return 'application/pdf' in response.headers.get('Content-Type', '')

def download_file(url, directory):
    response = requests.get(url)
    filename = os.path.join(directory, os.path.basename(urlparse(url).path))
    with open(filename, 'wb') as file:
        file.write(response.content)

def get_related_links_recursive(driver, base_url, url, excluded_keywords=[], visited_urls=set(), download_directory='downloads'):
    # Check if the page contains a PDF file
    if is_pdf(url):
        download_file(url, download_directory)
        return []

    # Navigate to the URL
    driver.get(url)
    time.sleep(2)  # Add a delay to ensure the page is loaded

    try:
        # Find the main content div
        main_content_div = driver.find_element(By.XPATH, "//div[@id='pnlMain' and @class='mainWrap']")

        # Find all relative links within the main content
        relative_links = main_content_div.find_elements(By.XPATH, ".//a[@href]")

        # Filter out specific links and make them absolute
        filtered_links = []
        for rel_link in relative_links:
            href = rel_link.get_attribute("href")
            absolute_link = urljoin(base_url, href)
            if absolute_link != base_url and not any(keyword in href for keyword in excluded_keywords):
                filtered_links.append(absolute_link)

        # Add the current URL to the set of visited URLs
        visited_urls.add(url)

        # Find and follow links to sub-pages
        sub_pages = [sub_page.get_attribute("href") for sub_page in relative_links if sub_page.get_attribute("href") not in visited_urls]
        for sub_page_url in sub_pages:
            if sub_page_url.startswith(base_url):
                filtered_links += get_related_links_recursive(driver, base_url, sub_page_url, excluded_keywords, visited_urls, download_directory)

        return filtered_links
    except NoSuchElementException:
        return []

# Set up the Chrome WebDriver
chrome_driver_path = '/Users/lmchak/llm/chromedriver-mac-x64/chromedriver'  # Replace with the actual path to your chromedriver
chrome_service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=chrome_service)

# Base URL
base_url = "https://www.ura.gov.sg/Corporate/Guidelines"

# Create a directory for downloads
download_directory = '/Users/lmchak/Downloads/URA'
os.makedirs(download_directory, exist_ok=True)

# Start link retrieval for the specified URL
ura_links = get_related_links_recursive(driver, base_url, base_url, download_directory=download_directory)

# Display the retrieved links
for i, link in enumerate(ura_links, start=1):
    print(f"{i}. {link}")

# Close the WebDriver
driver.quit()


KeyboardInterrupt: 

In [25]:
import os
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
import time
from urllib.parse import urljoin, urlparse

def is_pdf(url):
    # Send a HEAD request to get only the headers
    response = requests.head(url)

    # Check if the Content-Type header indicates a PDF file
    return 'application/pdf' in response.headers.get('Content-Type', '')

def download_file(url, directory):
    response = requests.get(url)
    filename = os.path.join(directory, os.path.basename(urlparse(url).path))
    with open(filename, 'wb') as file:
        file.write(response.content)

def download_page(url, directory):
    response = requests.get(url)
    filename = os.path.join(directory, os.path.basename(urlparse(url).path) + '.html')
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(response.text)

def get_related_links_recursive(driver, base_url, url, excluded_keywords=[], visited_urls=set(), download_directory='downloads'):
    # Check if the page contains a PDF file
    if is_pdf(url):
        download_file(url, download_directory)
        return []

    # Navigate to the URL
    driver.get(url)
    time.sleep(2)  # Add a delay to ensure the page is loaded

    try:
        # Find the main content div
        main_content_div = driver.find_element(By.XPATH, "//div[@id='pnlMain' and @class='mainWrap']")

        # Find all relative links within the main content
        relative_links = main_content_div.find_elements(By.XPATH, ".//a[@href]")

        # Filter out specific links and make them absolute
        filtered_links = []
        for rel_link in relative_links:
            href = rel_link.get_attribute("href")
            absolute_link = urljoin(base_url, href)
            if absolute_link != base_url and not any(keyword in href for keyword in excluded_keywords):
                filtered_links.append(absolute_link)

        # Add the current URL to the set of visited URLs
        visited_urls.add(url)

        # Download the current page
        download_page(url, download_directory)

        # Find and follow links to sub-pages
        sub_pages = [sub_page.get_attribute("href") for sub_page in relative_links if sub_page.get_attribute("href") not in visited_urls]
        for sub_page_url in sub_pages:
            if sub_page_url.startswith(base_url):
                filtered_links += get_related_links_recursive(driver, base_url, sub_page_url, excluded_keywords, visited_urls, download_directory)

        return filtered_links
    except NoSuchElementException:
        return []

# Set up the Chrome WebDriver
chrome_driver_path = '/Users/lmchak/llm/chromedriver-mac-x64/chromedriver'  # Replace with the actual path to your chromedriver
chrome_service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=chrome_service)

# Base URL
base_url = "https://www.ura.gov.sg/Corporate/Guidelines"

# Create a directory for downloads
download_directory = '/Users/lmchak/llm/URLdownload'
os.makedirs(download_directory, exist_ok=True)

# Start link retrieval for the specified URL
ura_links = get_related_links_recursive(driver, base_url, base_url, download_directory=download_directory)

# Display the retrieved links
for i, link in enumerate(ura_links, start=1):
    print(f"{i}. {link}")

# Close the WebDriver
driver.quit()


KeyboardInterrupt: 

In [28]:
import os
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
import time
from urllib.parse import urljoin, urlparse

def is_pdf(url):
    # Send a HEAD request with headers to get only the headers
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.head(url, headers=headers)

    # Check if the Content-Type header indicates a PDF file
    return 'application/pdf' in response.headers.get('Content-Type', '')

def download_file(url, directory):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers)
    filename = os.path.join(directory, os.path.basename(urlparse(url).path))
    with open(filename, 'wb') as file:
        file.write(response.content)

def download_page(url, directory):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers)
    filename = os.path.join(directory, os.path.basename(urlparse(url).path) + '.html')
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(response.text)

def get_related_links_recursive(driver, base_url, url, excluded_keywords=[], visited_urls=set(), download_directory='downloads'):
    # Check if the page contains a PDF file
    if is_pdf(url):
        download_file(url, download_directory)
        return []

    # Navigate to the URL
    driver.get(url)
    time.sleep(2)  # Add a delay to ensure the page is loaded

    try:
        # Find the main content div
        main_content_div = driver.find_element(By.XPATH, "//div[@id='pnlMain' and @class='mainWrap']")

        # Find all relative links within the main content
        relative_links = main_content_div.find_elements(By.XPATH, ".//a[@href]")

        # Filter out specific links and make them absolute
        filtered_links = []
        for rel_link in relative_links:
            href = rel_link.get_attribute("href")
            absolute_link = urljoin(base_url, href)
            if absolute_link != base_url and not any(keyword in href for keyword in excluded_keywords):
                filtered_links.append(absolute_link)

        # Add the current URL to the set of visited URLs
        visited_urls.add(url)

        # Download the current page
        download_page(url, download_directory)

        # Find and follow links to sub-pages
        sub_pages = [sub_page.get_attribute("href") for sub_page in relative_links if sub_page.get_attribute("href") not in visited_urls]
        for sub_page_url in sub_pages:
            if sub_page_url.startswith(base_url):
                filtered_links += get_related_links_recursive(driver, base_url, sub_page_url, excluded_keywords, visited_urls, download_directory)

        return filtered_links
    except NoSuchElementException:
        return []

# Set up the Chrome WebDriver
chrome_driver_path = '/Users/lmchak/llm/chromedriver-mac-x64/chromedriver'  # Replace with the actual path to your chromedriver
chrome_service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=chrome_service)

# Base URL
base_url = "https://www.ura.gov.sg/Corporate/Guidelines"

# Create a directory for downloads
download_directory = '/Users/lmchak/llm/URLdownload'
os.makedirs(download_directory, exist_ok=True)

# Start link retrieval for the specified URL
ura_links = get_related_links_recursive(driver, base_url, base_url, download_directory=download_directory)

# Display the retrieved links
for i, link in enumerate(ura_links, start=1):
    print(f"{i}. {link}")

# Close the WebDriver
driver.quit()


WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=122.0.6261.94)
Stacktrace:
0   chromedriver                        0x000000010b343e18 chromedriver + 4627992
1   chromedriver                        0x000000010b33bb43 chromedriver + 4594499
2   chromedriver                        0x000000010af39e4a chromedriver + 392778
3   chromedriver                        0x000000010af20164 chromedriver + 287076
4   chromedriver                        0x000000010af20053 chromedriver + 286803
5   chromedriver                        0x000000010af3c052 chromedriver + 401490
6   chromedriver                        0x000000010afc607b chromedriver + 966779
7   chromedriver                        0x000000010afa7613 chromedriver + 841235
8   chromedriver                        0x000000010af783da chromedriver + 648154
9   chromedriver                        0x000000010af78d1e chromedr

In [32]:
import os
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
import time
from urllib.parse import urljoin, urlparse

# Function to check if a URL points to a PDF file
def is_pdf(url):
    # Send a GET request and check if the response content indicates a PDF file
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers)

    # Check if the response content indicates a PDF file
    return 'application/pdf' in response.headers.get('Content-Type', '')

# Function to download a file (PDF or otherwise)
def download_file(url, directory):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers)
    
    # Get the file name from the URL and create the local file path
    filename = os.path.join(directory, os.path.basename(urlparse(url).path))
    
    # Check if the file already exists
    if os.path.exists(filename):
        # If the file already exists, append content to it
        with open(filename, 'ab') as file:
            file.write(response.content)
    else:
        # If the file does not exist, create it and write content
        with open(filename, 'wb') as file:
            file.write(response.content)

# Function to download an HTML page
def download_page(url, directory):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers)
    
    # Get the file name from the URL and create the local file path
    filename = os.path.join(directory, os.path.basename(urlparse(url).path) + '.html')
    
    # Check if the file already exists
    if os.path.exists(filename):
        # If the file already exists, append content to it
        with open(filename, 'a', encoding='utf-8') as file:
            file.write(response.text)
    else:
        # If the file does not exist, create it and write content
        with open(filename, 'w', encoding='utf-8') as file:
            file.write(response.text)

# Recursive function to get related links from a given URL
def get_related_links_recursive(driver, base_url, url, excluded_keywords=[], visited_urls=set(), download_directory='downloads'):
    # Check if the page contains a PDF file
    if is_pdf(url):
        download_file(url, download_directory)
        return []

    # Navigate to the URL
    driver.get(url)
    time.sleep(2)  # Add a delay to ensure the page is loaded

    try:
        # Find the main content div
        main_content_div = driver.find_element(By.XPATH, "//div[@id='pnlMain' and @class='mainWrap']")

        # Find all relative links within the main content
        relative_links = main_content_div.find_elements(By.XPATH, ".//a[@href]")

        # Filter out specific links and make them absolute
        filtered_links = []
        for rel_link in relative_links:
            href = rel_link.get_attribute("href")
            absolute_link = urljoin(base_url, href)
            if absolute_link != base_url and not any(keyword in href for keyword in excluded_keywords):
                filtered_links.append(absolute_link)

        # Add the current URL to the set of visited URLs
        visited_urls.add(url)

        # Download the current page
        download_page(url, download_directory)

        # Find and follow links to sub-pages
        sub_pages = [sub_page.get_attribute("href") for sub_page in relative_links if sub_page.get_attribute("href") not in visited_urls]
        for sub_page_url in sub_pages:
            if sub_page_url.startswith(base_url):
                filtered_links += get_related_links_recursive(driver, base_url, sub_page_url, excluded_keywords, visited_urls, download_directory)

        return filtered_links
    except NoSuchElementException:
        return []

# Set up the Chrome WebDriver
chrome_driver_path = '/Users/lmchak/llm/chromedriver-mac-x64/chromedriver'  # Replace with the actual path to your chromedriver
chrome_service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=chrome_service)

# Base URL
base_url = "https://www.ura.gov.sg/Corporate/Guidelines"

# Create a directory for downloads
download_directory = '/Users/lmchak/llm/URLdownload'
os.makedirs(download_directory, exist_ok=True)

# Start link retrieval for the specified URL
ura_links = get_related_links_recursive(driver, base_url, base_url, download_directory=download_directory)

# Display the retrieved links
for i, link in enumerate(ura_links, start=1):
    print(f"{i}. {link}")

# Close the WebDriver
driver.quit()


KeyboardInterrupt: 

In [38]:
import os
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
import time
from urllib.parse import urljoin, urlparse

# Function to check if a URL points to a PDF file
def is_pdf(url):
    # Send a GET request and check if the response content indicates a PDF file
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers)

    # Check if the response content indicates a PDF file
    return 'application/pdf' in response.headers.get('Content-Type', '')

# Function to download a file (PDF or otherwise)
def download_file(url, directory):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers)
    
    # Get the file name from the URL and create the local file path
    filename = os.path.join(directory, os.path.basename(urlparse(url).path))
    
    # Check if the file already exists
    if os.path.exists(filename):
        # If the file already exists, append content to it
        with open(filename, 'ab') as file:
            file.write(response.content)
    else:
        # If the file does not exist, create it and write content
        with open(filename, 'wb') as file:
            file.write(response.content)

# Function to download an HTML page
def download_page(url, directory):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers)
    
    # Get the file name from the URL and create the local file path
    filename = os.path.join(directory, os.path.basename(urlparse(url).path) + '.html')
    
    # Check if the file already exists
    if os.path.exists(filename):
        # If the file already exists, append content to it
        with open(filename, 'a', encoding='utf-8') as file:
            file.write(response.text)
    else:
        # If the file does not exist, create it and write content
        with open(filename, 'w', encoding='utf-8') as file:
            file.write(response.text)

# Recursive function to get related links from a given URL
def get_related_links_recursive(driver, base_url, url, excluded_keywords=[], visited_urls=set(), download_directory='downloads', depth=1):
    # Check if the page contains a PDF file
    if is_pdf(url):
        download_file(url, download_directory)
        return []

    # Navigate to the URL
    driver.get(url)
    time.sleep(2)  # Add a delay to ensure the page is loaded

    try:
        # Find the main content div
        main_content_div = driver.find_element(By.XPATH, "//div[@id='pnlMain' and @class='mainWrap']")

        # Find all relative links within the main content
        relative_links = main_content_div.find_elements(By.XPATH, ".//a[@href]")

        # Filter out specific links and make them absolute
        filtered_links = []
        for rel_link in relative_links:
            href = rel_link.get_attribute("href")
            absolute_link = urljoin(base_url, href)
            if absolute_link != base_url and not any(keyword in href for keyword in excluded_keywords):
                filtered_links.append(absolute_link)

        # Add the current URL to the set of visited URLs
        visited_urls.add(url)

        # Download the current page
        download_page(url, download_directory)

        # Find and follow links to sub-pages
        if depth > 1:
            sub_pages = [sub_page.get_attribute("href") for sub_page in relative_links if sub_page.get_attribute("href") not in visited_urls]
            for sub_page_url in sub_pages:
                if sub_page_url.startswith(base_url):
                    filtered_links += get_related_links_recursive(driver, base_url, sub_page_url, excluded_keywords, visited_urls, download_directory, depth-1)

        return filtered_links
    except NoSuchElementException:
        return []

# Set up the Chrome WebDriver
chrome_driver_path = '/Users/lmchak/llm/chromedriver-mac-x64/chromedriver'  # Replace with the actual path to your chromedriver
chrome_service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=chrome_service)

# Base URL
base_url = "https://www.ura.gov.sg/Corporate/Guidelines/Development-Control/Non-Residential"

# Create a directory for downloads
download_directory = '/Users/lmchak/llm/URLdownload'
os.makedirs(download_directory, exist_ok=True)

# Set the depth for crawling
depth = 5  # Specify the desired depth

# Start link retrieval for the specified URL
ura_links = get_related_links_recursive(driver, base_url, base_url, download_directory=download_directory, depth=depth)

# Display the retrieved links
for i, link in enumerate(ura_links, start=1):
    print(f"{i}. {link}")

# Close the WebDriver
driver.quit()


KeyboardInterrupt: 

In [40]:
import os
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
import time
from urllib.parse import urljoin, urlparse

# Function to check if a URL points to a PDF file
def is_pdf(url):
    # Send a GET request and check if the response content indicates a PDF file
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers)

    # Check if the response content indicates a PDF file
    return 'application/pdf' in response.headers.get('Content-Type', '')

# Function to download a file (PDF or otherwise)
def download_file(url, directory):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers)
    
    # Get the file name from the URL and create the local file path
    filename = os.path.join(directory, os.path.basename(urlparse(url).path))
    
    # Check if the file already exists
    index = 1
    while os.path.exists(filename):
        # If the file already exists, create a new filename with an incremental index
        filename = os.path.join(directory, f"{os.path.splitext(os.path.basename(urlparse(url).path))[0]}_{index}.pdf")
        index += 1
    
    # Write content to the file
    with open(filename, 'wb') as file:
        file.write(response.content)

# Function to download an HTML page
def download_page(url, directory):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers)
    
    # Get the file name from the URL and create the local file path
    filename = os.path.join(directory, os.path.basename(urlparse(url).path) + '.html')
    
    # Check if the file already exists
    index = 1
    while os.path.exists(filename):
        # If the file already exists, create a new filename with an incremental index
        filename = os.path.join(directory, f"{os.path.splitext(os.path.basename(urlparse(url).path))[0]}_{index}.html")
        index += 1
    
    # Write content to the file
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(response.text)

# Recursive function to get related links from a given URL
def get_related_links_recursive(driver, base_url, url, excluded_keywords=[], visited_urls=set(), download_directory='downloads', depth=1):
    # Check if the page contains a PDF file
    if is_pdf(url):
        download_file(url, download_directory)
        return []

    # Navigate to the URL
    driver.get(url)
    time.sleep(2)  # Add a delay to ensure the page is loaded

    try:
        # Find the main content div
        main_content_div = driver.find_element(By.XPATH, "//div[@id='pnlMain' and @class='mainWrap']")

        # Find all relative links within the main content
        relative_links = main_content_div.find_elements(By.XPATH, ".//a[@href]")

        # Filter out specific links and make them absolute
        filtered_links = []
        for rel_link in relative_links:
            href = rel_link.get_attribute("href")
            absolute_link = urljoin(base_url, href)
            if absolute_link != base_url and not any(keyword in href for keyword in excluded_keywords):
                filtered_links.append(absolute_link)

        # Add the current URL to the set of visited URLs
        visited_urls.add(url)

        # Download the current page
        download_page(url, download_directory)

        # Find and follow links to sub-pages
        if depth > 1:
            sub_pages = [sub_page.get_attribute("href") for sub_page in relative_links if sub_page.get_attribute("href") not in visited_urls]
            for sub_page_url in sub_pages:
                if sub_page_url.startswith(base_url):
                    filtered_links += get_related_links_recursive(driver, base_url, sub_page_url, excluded_keywords, visited_urls, download_directory, depth-1)

        return filtered_links
    except NoSuchElementException:
        return []

# Set up the Chrome WebDriver
chrome_driver_path = '/Users/lmchak/llm/chromedriver-mac-x64/chromedriver'  # Replace with the actual path to your chromedriver
chrome_service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=chrome_service)

# Base URL
base_url = "https://www.ura.gov.sg/Corporate/Guidelines/Development-Control/Non-Residential"

# Create a directory for downloads
download_directory = '/Users/lmchak/llm/URLdownload'
os.makedirs(download_directory, exist_ok=True)

# Set the depth for crawling
depth = 5  # Specify the desired depth

# Start link retrieval for the specified URL
ura_links = get_related_links_recursive(driver, base_url, base_url, download_directory=download_directory, depth=depth)

# Display the retrieved links
for i, link in enumerate(ura_links, start=1):
    print(f"{i}. {link}")

# Close the WebDriver
driver.quit()


KeyboardInterrupt: 

In [43]:
import os
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
import time
from urllib.parse import urljoin, urlparse

# Function to check if a URL points to a PDF file
def is_pdf(url):
    # Send a GET request and check if the response content indicates a PDF file
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers)

    # Check if the response content indicates a PDF file
    return 'application/pdf' in response.headers.get('Content-Type', '')

# Function to download a file (PDF or otherwise)
def download_file(url, directory):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers)
    
    # Get the file name from the URL and create the local file path
    filename = os.path.join(directory, os.path.basename(urlparse(url).path))
    
    # Check if the file already exists
    index = 1
    while os.path.exists(filename):
        # If the file already exists, create a new filename with an incremental index
        filename = os.path.join(directory, f"{os.path.splitext(os.path.basename(urlparse(url).path))[0]}_{index}.pdf")
        index += 1
    
    # Write content to the file
    with open(filename, 'wb') as file:
        file.write(response.content)

# Function to download an HTML page
def download_page(url, directory):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers)
    
    # Get the file name from the URL and create the local file path
    filename = os.path.join(directory, os.path.basename(urlparse(url).path) + '.html')
    
    # Check if the file already exists
    index = 1
    while os.path.exists(filename):
        # If the file already exists, create a new filename with an incremental index
        filename = os.path.join(directory, f"{os.path.splitext(os.path.basename(urlparse(url).path))[0]}_{index}.html")
        index += 1
    
    # Write content to the file
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(response.text)

# Recursive function to get related links from a given URL
def get_related_links_recursive(driver, base_url, url, excluded_keywords=[], visited_urls=set(), download_directory='downloads', depth=1):
    # Check if the page contains a PDF file
    if is_pdf(url):
        download_file(url, download_directory)
        return []

    # Navigate to the URL
    driver.get(url)
    time.sleep(2)  # Add a delay to ensure the page is loaded

    try:
        # Find the main content div
        main_content_div = driver.find_element(By.XPATH, "//div[@id='pnlMain' and @class='mainWrap']")

        # Find all relative links within the main content
        relative_links = main_content_div.find_elements(By.XPATH, ".//a[@href]")

        # Filter out specific links and make them absolute
        filtered_links = []
        for rel_link in relative_links:
            href = rel_link.get_attribute("href")
            absolute_link = urljoin(base_url, href)
            if absolute_link != base_url and not any(keyword in href for keyword in excluded_keywords):
                filtered_links.append(absolute_link)

        # Add the current URL to the set of visited URLs
        visited_urls.add(url)

        # Download the current page
        download_page(url, download_directory)

        # Find and follow links to sub-pages
        if depth > 1:
            sub_pages = [sub_page.get_attribute("href") for sub_page in relative_links if sub_page.get_attribute("href") not in visited_urls]
            for sub_page_url in sub_pages:
                if sub_page_url.startswith(base_url):
                    filtered_links += get_related_links_recursive(driver, base_url, sub_page_url, excluded_keywords, visited_urls, download_directory, depth-1)

        return filtered_links
    except NoSuchElementException:
        return []

# Set up the Chrome WebDriver
chrome_driver_path = '/Users/lmchak/llm/chromedriver-mac-x64/chromedriver'  # Replace with the actual path to your chromedriver
chrome_service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=chrome_service)

# Base URL
base_url = "https://www.ura.gov.sg/Corporate/Guidelines/Development-Control/Non-Residential"

# Create a directory for downloads
download_directory = '/Users/lmchak/llm/URLdownload'
os.makedirs(download_directory, exist_ok=True)

# Set the depth for crawling
depth = 5  # Specify the desired depth

# Start link retrieval for the specified URL
ura_links = get_related_links_recursive(driver, base_url, base_url, download_directory=download_directory, depth=depth)

# Display the retrieved links
for i, link in enumerate(ura_links, start=1):
    print(f"{i}. {link}")

# Close the WebDriver
driver.quit()


WebDriverException: Message: disconnected: Unable to receive message from renderer
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=122.0.6261.94)
Stacktrace:
0   chromedriver                        0x00000001051ede18 chromedriver + 4627992
1   chromedriver                        0x00000001051e5b43 chromedriver + 4594499
2   chromedriver                        0x0000000104de3e4a chromedriver + 392778
3   chromedriver                        0x0000000104dcab3d chromedriver + 289597
4   chromedriver                        0x0000000104dcaa2b chromedriver + 289323
5   chromedriver                        0x0000000104dc97d8 chromedriver + 284632
6   chromedriver                        0x0000000104dc9b6a chromedriver + 285546
7   chromedriver                        0x0000000104de665e chromedriver + 403038
8   chromedriver                        0x0000000104e71275 chromedriver + 971381
9   chromedriver                        0x0000000104e

In [48]:
import os
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
import time
from urllib.parse import urljoin, urlparse

# Function to check if a URL points to a PDF file
def is_pdf(url):
    # Send a GET request and check if the response content indicates a PDF file
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers)

    # Check if the response content indicates a PDF file
    return 'application/pdf' in response.headers.get('Content-Type', '')

# Function to download a file (PDF or otherwise)
def download_file(url, directory, depth_index):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers)
    
    # Get the file name from the URL and create the local file path
    filename = os.path.join(directory, f"{os.path.basename(urlparse(url).path)}_{depth_index}.pdf")
    
    # Check if the file already exists
    index = 1
    while os.path.exists(filename):
        # If the file already exists, create a new filename with an incremental index
        filename = os.path.join(directory, f"{os.path.splitext(os.path.basename(urlparse(url).path))[0]}_{depth_index}_{index}.pdf")
        index += 1
    
    # Write content to the file
    with open(filename, 'wb') as file:
        file.write(response.content)

# Function to download an HTML page
def download_page(url, directory, depth_index):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers)
    
    # Get the file name from the URL and create the local file path
    filename = os.path.join(directory, f"{os.path.basename(urlparse(url).path)}_{depth_index}.html")
    
    # Check if the file already exists
    index = 1
    while os.path.exists(filename):
        # If the file already exists, create a new filename with an incremental index
        filename = os.path.join(directory, f"{os.path.splitext(os.path.basename(urlparse(url).path))[0]}_{depth_index}_{index}.html")
        index += 1
    
    # Write content to the file
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(response.text)

# Recursive function to get related links from a given URL
def get_related_links_recursive(driver, base_url, url, excluded_keywords=[], visited_urls=set(), download_directory='downloads', depth=1):
    # Check if the page contains a PDF file
    if is_pdf(url):
        download_file(url, download_directory, depth)
        return []

    # Navigate to the URL
    driver.get(url)
    time.sleep(2)  # Add a delay to ensure the page is loaded

    try:
        # Find the main content div
        main_content_div = driver.find_element(By.XPATH, "//div[@id='pnlMain' and @class='mainWrap']")

        # Find all relative links within the main content
        relative_links = main_content_div.find_elements(By.XPATH, ".//a[@href]")

        # Filter out specific links and make them absolute
        filtered_links = []
        for rel_link in relative_links:
            href = rel_link.get_attribute("href")
            absolute_link = urljoin(base_url, href)
            if absolute_link != base_url and not any(keyword in href for keyword in excluded_keywords):
                filtered_links.append(absolute_link)

        # Add the current URL to the set of visited URLs
        visited_urls.add(url)

        # Download the current page
        download_page(url, download_directory, depth)

        # Find and follow links to sub-pages
        if depth > 1:
            sub_pages = [sub_page.get_attribute("href") for sub_page in relative_links if sub_page.get_attribute("href") not in visited_urls]
            for sub_page_url in sub_pages:
                if sub_page_url.startswith(base_url):
                    filtered_links += get_related_links_recursive(driver, base_url, sub_page_url, excluded_keywords, visited_urls, download_directory, depth=depth-1)

        return filtered_links
    except NoSuchElementException:
        return []

# Set up the Chrome WebDriver
chrome_driver_path = '/Users/lmchak/llm/chromedriver-mac-x64/chromedriver'  # Replace with the actual path to your chromedriver
chrome_service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=chrome_service)

# Base URL
base_url = "https://www.ura.gov.sg/Corporate/Guidelines/Development-Control/Non-Residential"

# Create a directory for downloads
download_directory = '/Users/lmchak/llm/URLdownload'
os.makedirs(download_directory, exist_ok=True)

# Set the depth for crawling
depth = 3  # Specify the desired depth

# Start link retrieval for the specified URL
ura_links = get_related_links_recursive(driver, base_url, base_url, download_directory=download_directory, depth=depth)

# Display the retrieved links
for i, link in enumerate(ura_links, start=1):
    print(f"{i}. {link}")

# Close the WebDriver
driver.quit()


KeyboardInterrupt: 

In [51]:
import os
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
import time
from urllib.parse import urljoin, urlparse
from bs4 import BeautifulSoup

# Function to check if a URL points to a PDF file
def is_pdf(url):
    # Send a GET request and check if the response content indicates a PDF file
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers)

    # Check if the response content indicates a PDF file
    return 'application/pdf' in response.headers.get('Content-Type', '')

# Function to download a file (PDF or otherwise)
def download_file(url, title, directory, depth):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers)
    
    # Get the file name from the URL and create the local file path
    filename = os.path.join(directory, f"{title}_depth_{depth}_{os.path.basename(urlparse(url).path)}")
    
    # Check if the file already exists
    index = 1
    while os.path.exists(filename):
        # If the file already exists, create a new filename with an incremental index
        filename = os.path.join(directory, f"{title}_depth_{depth}_{os.path.splitext(os.path.basename(urlparse(url).path))[0]}_{index}.pdf")
        index += 1
    
    # Write content to the file
    with open(filename, 'wb') as file:
        file.write(response.content)

# Function to download an HTML page
def download_page(url, title, directory, depth):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers)
    
    # Use BeautifulSoup to extract the title from the HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    page_title = soup.find('span', {'id': 'column_0_main_1_txtContentTitle'})
    
    # Use the extracted title or a default if not found
    title_to_use = page_title.text.strip() if page_title else 'Untitled'
    
    # Get the file name from the URL and create the local file path
    filename = os.path.join(directory, f"{title_to_use}_depth_{depth}_{os.path.basename(urlparse(url).path)}.html")
    
    # Check if the file already exists
    index = 1
    while os.path.exists(filename):
        # If the file already exists, create a new filename with an incremental index
        filename = os.path.join(directory, f"{title_to_use}_depth_{depth}_{os.path.splitext(os.path.basename(urlparse(url).path))[0]}_{index}.html")
        index += 1
    
    # Write content to the file
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(response.text)

# Recursive function to get related links from a given URL
def get_related_links_recursive(driver, base_url, url, excluded_keywords=[], visited_urls=set(), download_directory='downloads', depth=1):
    # Check if the page contains a PDF file
    if is_pdf(url):
        # Extract the title from the PDF page (if applicable)
        response = requests.get(url)
        title = f"pdf_title_{depth}"
        download_file(url, title, download_directory, depth)
        return []

    # Navigate to the URL
    driver.get(url)
    time.sleep(2)  # Add a delay to ensure the page is loaded

    try:
        # Find the main content div
        main_content_div = driver.find_element(By.XPATH, "//div[@id='pnlMain' and @class='mainWrap']")

        # Find all relative links within the main content
        relative_links = main_content_div.find_elements(By.XPATH, ".//a[@href]")

        # Filter out specific links and make them absolute
        filtered_links = []
        for rel_link in relative_links:
            href = rel_link.get_attribute("href")
            absolute_link = urljoin(base_url, href)
            if absolute_link != base_url and not any(keyword in href for keyword in excluded_keywords):
                filtered_links.append(absolute_link)

        # Add the current URL to the set of visited URLs
        visited_urls.add(url)

        # Download the current page
        download_page(url, base_url, download_directory, depth)

        # Find and follow links to sub-pages
        if depth > 1:
            sub_pages = [sub_page.get_attribute("href") for sub_page in relative_links if sub_page.get_attribute("href") not in visited_urls]
            for sub_page_url in sub_pages:
                if sub_page_url.startswith(base_url):
                    filtered_links += get_related_links_recursive(driver, base_url, sub_page_url, excluded_keywords, visited_urls, download_directory, depth=depth-1)

        return filtered_links
    except NoSuchElementException:
        return []

# Set up the Chrome WebDriver
chrome_driver_path = '/Users/lmchak/llm/chromedriver-mac-x64/chromedriver'  # Replace with the actual path to your chromedriver
chrome_service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=chrome_service)

# Base URL
base_url = "https://www.ura.gov.sg/Corporate/Guidelines/Development-Control/Non-Residential"

# Create a directory for downloads
download_directory = '/Users/lmchak/llm/URLdownload'
os.makedirs(download_directory, exist_ok=True)

# Set the depth for crawling
depth = 4  # Specify the desired depth

# Start link retrieval for the specified URL
ura_links = get_related_links_recursive(driver, base_url, base_url, download_directory=download_directory, depth=depth)

# Display the retrieved links
for i, link in enumerate(ura_links, start=1):
    print(f"{i}. {link}")

# Close the WebDriver
driver.quit()


KeyboardInterrupt: 